<a href="https://colab.research.google.com/github/b09702043/PTT_crawler/blob/main/%E5%AD%B8%E5%B9%B4%E7%B4%80%E9%8C%84%E5%90%88%E4%BD%B5%E6%AA%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [367]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [368]:
import os
path = "/content/drive/MyDrive/比賽紀錄"  # 手動更改存取雲端位置
os.chdir(path)

In [369]:
# 最後更新時間
from datetime import datetime
import pytz

# 設置所需的時區
desired_timezone = pytz.timezone('Asia/Taipei')

# 取得當前時間並轉換為所需時區
now = datetime.now(desired_timezone)

print("最後更新時間 :", now.strftime("%Y-%m-%d %H:%M:%S"))

最後更新時間 : 2024-04-18 21:01:50


# 使用說明

*   檢視者 : 請往最左邊的工具列，打開最上面第一個圖示「目錄」，點擊有興趣的項目即可
*   編輯者 : 只要紀錄表格式不要更動(尤其前十列的列數不要變)，這個程式碼就跑得動


# 前置作業

1. 檢查要算年度紀錄的工作表的google sheet中**「姓名」該列不要有函數**，請以值顯示 ( *複製-僅貼上值*；因該函數是連結雲端的表，後續可能會變空值 )
2. 把要算年度紀錄的工作表的google sheet下載成 **'.xlsx'** 檔，重新上傳至**「比賽紀錄」資料夾**
3. 把要算年度紀錄的工作表名稱以string填入下方程式碼的 record_list，填入時建議**利用 「重新命名」 完整複製過來** (有時有出乎意料的空格)



In [370]:
# 把要算年度紀錄的工作表名稱以string填入
record_list = ['2024聯盟賽 .xlsx', '2024台大盃.xlsx']

# 程式碼

In [371]:
import pandas as pd
import numpy as np

In [372]:
# 背號對照:分成OB和現役(type:dict)，可手動更新
# 請注意單名比較特別
OB_player = {
        34: "呂俊緯",
        13: "董逢霖",
        24: "徐兆詮"
}
active_player = {
        11: "郭冠廷",
        40: "陳詠澤",
        99: "林柏論",
        3: "陳 諒",
        31: "藍正倫",
        26: "吳旻峻",
        5: "黃式傑",
        1: "趙澂偉",
        48: "吳文嘉",
        56: "莊樹人",
        18: "張恒易",
        17: "林彥均",
        19: "胡鈞捷"
}

In [373]:
# 將背號對照表key和value對換，方便後面總表操作
# 合併成一個新字典，並只抽出人名放進player_list
OB_player_flipped = {v: k for k, v in OB_player.items()}
active_player_flipped = {v: k for k, v in active_player.items()}
player_dict_flipped = {**OB_player_flipped, **active_player_flipped}
player_list = list(player_dict_flipped.keys())
player_list = [player.strip() for player in player_list]
player_list

['呂俊緯',
 '董逢霖',
 '徐兆詮',
 '郭冠廷',
 '陳詠澤',
 '林柏論',
 '陳 諒',
 '藍正倫',
 '吳旻峻',
 '黃式傑',
 '趙澂偉',
 '吳文嘉',
 '莊樹人',
 '張恒易',
 '林彥均',
 '胡鈞捷']

In [374]:
# 設定野手數據標題列 : 背號 姓名 打席 打數 安打 得分 打點 三振 保送 失誤
batter_final_data = {'背號': [],
            '姓名': [],
            '打席': [],
            '打數': [],
            '安打': [],
            '得分': [],
            '打點': [],
            '三振': [],
            '保送': [],
            '失誤': [],
}
batter_final_df = pd.DataFrame(batter_final_data)
batter_final_df

,背號,姓名,打席,打數,安打,得分,打點,三振,保送,失誤


In [375]:
# 設定投手數據標題列 : 背號 姓名 局數 人次 失分 責失 三振 四壞
pitcher_final_data = {'背號': [],
            '姓名': [],
            '局數': [],
            '人次': [],
            '失分': [],
            '責失': [],
            '三振': [],
            '四壞': [],
}
pitcher_final_df = pd.DataFrame(pitcher_final_data)
pitcher_final_df

,背號,姓名,局數,人次,失分,責失,三振,四壞


In [376]:
hit_final_data = {'一安': [],
          '二安': [],
          '三安': [],
          '全壘打': [],
          '犧牲打': [],
}
hit_final_df = pd.DataFrame(hit_final_data)
hit_final_df

,一安,二安,三安,全壘打,犧牲打


In [377]:
# 產生總表
batter_dfs = []
pitcher_dfs = []
hit_dfs = []

for file in record_list:
    xls = pd.ExcelFile(file)

    # 獲取工作表名稱列表
    sheet_names = xls.sheet_names

    # 遍歷每個工作表
    for sheet_name in sheet_names:
      if sheet_name != '背號對照':

        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
        #print(df.shape)

        # 找野手和投手數據的「起始列」索引值
        batter_row_index = int(df[df.iloc[:, 4] == '打席'].index[0])
        pitcher_row_index = int(df[df.iloc[:, 2] == '投手'].index[0])
        #print(batter_row_index)
        #print(batter_row_index)

        # 找野手和投手數據的「終止欄」索引值
        # 使用 isin() 方法檢查特定值是否存在於每一欄
        # 傳回一個布林索引的 Series，其中 True 表示該欄位包含特定值
        contains_batter_value = df.isin(['失誤']).any(axis=0)
        contains_pitcher_value = df.isin(['四壞']).any(axis=0)

        # 提取含有特定值的欄索引
        batter_column_index = int(contains_batter_value[contains_batter_value].index[0])
        pitcher_column_index = int(contains_pitcher_value[contains_pitcher_value].index[0])

        # 分別存野手和投手的dataframe
        batter_df = df.iloc[batter_row_index:(pitcher_row_index-1), :(batter_column_index+1)]
        pitcher_df = df.iloc[pitcher_row_index:, 1:(pitcher_column_index+1)]

        # 重設列的編號
        batter_df.reset_index(drop=True, inplace=True)
        pitcher_df.reset_index(drop=True, inplace=True)

        # 把第一列設為標題，之後刪除第一列
        batter_df.columns = batter_df.iloc[0]
        batter_df = batter_df.iloc[1:]
        pitcher_df.columns = pitcher_df.iloc[0]
        pitcher_df = pitcher_df.iloc[1:]

        # 不要的欄刪掉(背號有時候亂穿就拿掉了)
        batter_df.drop(columns=['棒次', '守備位置', "背號"], inplace=True)
        pitcher_df.drop(columns=['投手'], inplace=True)
        #batter_df = batter_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        #pitcher_df = pitcher_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

        # 處理安打、全壘打、犧牲打細項
        hit_df = df.iloc[10:(batter_row_index-1), :]
        hit_df_for_apply = hit_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

        # 複製一個避免apply出問題
        hit_df_copy = hit_df_for_apply.copy()
        hit_df_copy.columns = hit_df_copy.iloc[0]
        hit_df_copy = hit_df_copy.iloc[1:]

        hit_df_copy['一安'] = hit_df_for_apply.iloc[:, 4:].apply(lambda row: row.tolist().count('一安'), axis=1).astype(int)
        hit_df_copy['二安'] = hit_df_for_apply.iloc[:, 4:].apply(lambda row: row.tolist().count('二安'), axis=1).astype(int)
        hit_df_copy['三安'] = hit_df_for_apply.iloc[:, 4:].apply(lambda row: row.tolist().count('三安'), axis=1).astype(int)
        hit_df_copy['全壘打'] = hit_df_for_apply.iloc[:, 4:].apply(lambda row: row.tolist().count('全壘打'), axis=1).astype(int)
        hit_df_copy['犧牲打'] = hit_df_for_apply.iloc[:, 4:].apply(lambda row: row.str.contains('犧').sum(), axis=1).astype(int)
        hit_df_copy = hit_df_copy.drop(columns=hit_df_copy.columns.difference(['姓名', '一安', '二安', '三安', '全壘打', '犧牲打']))

        batter_dfs.append(batter_df)
        pitcher_dfs.append(pitcher_df)
        hit_dfs.append(hit_df_copy)
        #print(batter_df)
        #print(pitcher_df)
        #print(hit_df_copy)

In [378]:
batter_final_df = pd.concat(batter_dfs, ignore_index=True)

# 將所有不同選手的數據進行合併
batter_final_df = batter_final_df.groupby(['姓名']).agg({'打席': sum,
                            '打數': sum,
                            '安打': sum,
                            '得分': sum,
                            '打點': sum,
                            '三振': sum,
                            '保送': sum,
                            '失誤': sum,
}).reset_index()
#batter_final_df

In [379]:
hit_final_df = pd.concat(hit_dfs, ignore_index=True)

# 將所有不同選手的數據進行合併
hit_final_df = hit_final_df.groupby(['姓名']).agg({'一安': sum,
                          '二安': sum,
                          '三安': sum,
                          '全壘打': sum,
                          '犧牲打': sum,
}).reset_index()
#hit_final_df

# 打擊表現

## 總表

In [380]:
final_data = pd.merge(batter_final_df, hit_final_df)

final_data['打擊率'] = final_data['安打']/final_data['打數']
final_data['上壘率'] = (final_data['安打']+final_data['保送']) / (final_data['打數']+final_data['保送']+final_data['犧牲打'])
final_data['長打率'] = (final_data['一安']*1 + final_data['二安']*2 + final_data['三安']*3 + final_data['全壘打']*4)/final_data['打數']
final_data['攻擊指數'] = final_data['上壘率'] + final_data['長打率']
final_data['純長打率'] = final_data['長打率'] - final_data['打擊率']

# 添加背號列
final_data["背號"] = final_data["姓名"].map(player_dict_flipped.get).fillna(np.nan)

# 將非空值轉換為整數
final_data["背號"] = final_data["背號"].fillna(-1).astype(int)

# 新增一列名為 "status"，根據球員是否在 OB_player 或 active_player 中進行設置
final_data['status'] = final_data['背號'].apply(lambda x: 'OB' if x in OB_player else 'active')

final_data.set_index("背號", inplace=True)

final_data

,姓名,打席,打數,安打,得分,打點,三振,保送,失誤,一安,二安,三安,全壘打,犧牲打,打擊率,上壘率,長打率,攻擊指數,純長打率,status
背號,,,,,,,,,,,,,,,,,,,,
48,吳文嘉,13,11,2,3,4,2,2,3,2,0,0,0,0,0.2,0.3,0.2,0.5,0.0,active
26,吳旻峻,11,10,2,1,4,1,1,0,0,2,0,0,0,0.2,0.3,0.4,0.7,0.2,active
34,呂俊緯,4,2,1,1,0,0,2,1,0,1,0,0,0,0.5,0.8,1.0,1.8,0.5,OB
-1,呂冠毅,6,5,1,1,1,0,1,1,1,0,0,0,0,0.2,0.3,0.2,0.5,0.0,active
18,張恒易,20,16,8,8,4,0,3,2,5,0,3,0,1,0.5,0.6,0.9,1.4,0.4,active
24,徐兆詮,2,1,1,1,1,0,1,0,1,0,0,0,0,1.0,1.0,1.0,2.0,0.0,OB
17,林彥均,17,13,6,7,4,1,4,5,4,0,1,1,0,0.5,0.6,0.8,1.4,0.4,active
99,林柏論,8,6,2,2,2,0,2,0,2,0,0,0,0,0.3,0.5,0.3,0.8,0.0,active
19,胡鈞捷,5,5,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,active


## 排行榜
目前都是用總表去做排名，並無區分OB或現役
*   顯示前幾名可於 **head()** 中設定，填入想要呈現的人數即可
*   於下方程式碼新增兩個分別為純OB和純現役的dataframe，若只想知道純現役可手動更改程式

如下範例 :

hit = **final_data**.sort_values(by='安打', ascending=False).head(5)

改成

hit = **active_players_df**.sort_values(by='安打', ascending=False).head(5)







In [381]:
# 純OB和純現役的dataframe
OB_players_df = final_data[final_data['status'] == 'OB']
active_players_df = final_data[final_data['status'] == 'active']

### 安打

In [382]:
hit = final_data.sort_values(by='安打', ascending=False).head(5)
hit[['姓名', '安打']]

,姓名,安打
背號,,
5,黃式傑,9
18,張恒易,8
3,陳 諒,6
17,林彥均,6
1,趙澂偉,4


### 打點

In [383]:
point = final_data.sort_values(by='打點', ascending=False).head(5)
point[['姓名', '打點']]

,姓名,打點
背號,,
5,黃式傑,13
13,董逢霖,5
17,林彥均,4
1,趙澂偉,4
26,吳旻峻,4


### 三振

In [384]:
K = final_data.sort_values(by='三振', ascending=False).head(5)
K[['姓名', '三振']]

,姓名,三振
背號,,
48,吳文嘉,2
1,趙澂偉,2
56,莊樹人,2
40,陳詠澤,1
17,林彥均,1


### 保送

In [385]:
BB = final_data.sort_values(by='保送', ascending=False).head(5)
BB[['姓名', '保送']]

,姓名,保送
背號,,
40,陳詠澤,4
17,林彥均,4
18,張恒易,3
48,吳文嘉,2
1,趙澂偉,2


### 全壘打

In [386]:
home_run = final_data.sort_values(by='全壘打', ascending=False).head(3)
home_run[['姓名', '全壘打']]

,姓名,全壘打
背號,,
5,黃式傑,3
13,董逢霖,1
17,林彥均,1


### 失誤

In [387]:
error = final_data.sort_values(by='失誤', ascending=False).head(5)
error[['姓名', '失誤']]

,姓名,失誤
背號,,
1,趙澂偉,6
17,林彥均,5
48,吳文嘉,3
18,張恒易,2
5,黃式傑,2


### 打擊率

In [388]:
pd.set_option('display.float_format', '{:.3f}'.format)  # 設定顯示小數後三碼

batting_avg = final_data.sort_values(by='打擊率', ascending=False).head(5)
batting_avg[['姓名', '打擊率']]

,姓名,打擊率
背號,,
24,徐兆詮,1.000
13,董逢霖,0.600
5,黃式傑,0.529
34,呂俊緯,0.500
18,張恒易,0.500


### 上壘率

In [389]:
on_base_percentage = final_data.sort_values(by='上壘率', ascending=False).head(5)
on_base_percentage[['姓名', '上壘率']]

,姓名,上壘率
背號,,
24,徐兆詮,1.000
34,呂俊緯,0.750
13,董逢霖,0.667
40,陳詠澤,0.667
11,郭冠廷,0.600


### 長打率

In [390]:
slugging_percentage = final_data.sort_values(by='長打率', ascending=False).head(5)
slugging_percentage[['姓名', '長打率']]

,姓名,長打率
背號,,
5,黃式傑,1.412
13,董逢霖,1.400
34,呂俊緯,1.000
24,徐兆詮,1.000
18,張恒易,0.875


### 攻擊指數(OPS)

In [391]:
OPS = final_data.sort_values(by='攻擊指數', ascending=False).head(5)
OPS[['姓名', '攻擊指數']]

,姓名,攻擊指數
背號,,
13,董逢霖,2.067
24,徐兆詮,2.000
5,黃式傑,1.938
34,呂俊緯,1.750
17,林彥均,1.434


### 純長打率

In [392]:
pure_slugging_percentage = final_data.sort_values(by='純長打率', ascending=False).head(5)
pure_slugging_percentage[['姓名', '純長打率']]

,姓名,純長打率
背號,,
5,黃式傑,0.882
13,董逢霖,0.800
34,呂俊緯,0.500
17,林彥均,0.385
18,張恒易,0.375


## 圖表
還沒想好要畫啥

# 投手表現

In [393]:
pitcher_final_df = pd.concat(pitcher_dfs, ignore_index=True)
pitcher_final_df['背號'] = pitcher_final_df['背號'].str.lstrip('#')
pitcher_final_df['背號'] = pitcher_final_df['背號'].astype(int)  # 將背號欄轉換為整數以進行排序
pitcher_final_df = pitcher_final_df.sort_values(by='背號').reset_index(drop=True)  # 按背號大小排序並重新設置索引

# 添加背號列
pitcher_final_df["背號"] = pitcher_final_df["姓名"].map(player_dict_flipped.get).fillna(np.nan)

# 將非空值轉換為整數
pitcher_final_df["背號"] = pitcher_final_df["背號"].fillna(-1).astype(int)

# 新增一列名為 "status"，根據球員是否在 OB_player 或 active_player 中進行設置
pitcher_final_df['status'] = pitcher_final_df['背號'].apply(lambda x: 'OB' if x in OB_player else 'active')

#pitcher_final_df.set_index("背號", inplace=True)

pd.set_option('display.float_format', '{:.1f}'.format)  # 設定只顯示小數後三碼

pitcher_final_df

,背號,姓名,局數,人次,失分,責失,三振,四壞,status
0,3,陳 諒,2.2,24,10,10,1,10,active
1,3,陳 諒,0,3,0,0,0,2,active
2,3,陳 諒,2,22,11,11,1,11,active
3,3,陳 諒,2.1,26,16,3,1,9,active
4,5,黃式傑,2,11,2,1,2,2,active
5,5,黃式傑,6,38,15,12,3,4,active
6,5,黃式傑,1.2,7,2,0,0,0,active
7,40,陳詠澤,0.1,2,0,0,0,1,active
8,40,陳詠澤,5,29,4,2,4,2,active
9,40,陳詠澤,4,17,3,3,2,3,active


In [394]:
# 將所有不同選手的數據進行合併
pitcher_final_df = pitcher_final_df.groupby(['背號', '姓名']).agg({
                                    '局數': sum,
                                    '失分': sum,
                                    '責失': sum,
                                    '三振': sum,
                                    '四壞': sum,
})
pitcher_final_df['自責分率'] = (pitcher_final_df['責失']*7) / pitcher_final_df['局數']
pitcher_final_df

,,局數,失分,責失,三振,四壞,自責分率
背號,姓名,,,,,,
3,陳 諒,6.3,37,24,3,32,26.7
5,黃式傑,9.2,19,13,5,6,9.9
40,陳詠澤,9.1,7,5,6,6,3.8
